In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]
results_dict = {}
for r in results: # S
    model_name, dataset_name, metric_name, metric_value = r
    key = dataset_name
    subkey = metric_name
    subsubkey = model_name
    values = list(set(r) - set([key, subkey, subsubkey]))
    subsubkey = '-'.join(subsubkey.split('-')[2:])
    if key not in results_dict:
        results_dict[key] = {} # .append(values)
    # else:
    #     results_dict[key] = {} #[values]
    if subkey not in results_dict[key]:
        results_dict[key][subkey] = {}
    if subsubkey in results_dict[key][subkey]:
        results_dict[key][subkey][subsubkey] += values
    else:
        results_dict[key][subkey][subsubkey] = values

# results_dict

In [3]:
def remove_underscore(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]

df_2 = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df_2['value'] = pd.to_numeric(df_2['value'])
df_2['model'] = df_2['model'].map(lambda x: '-'.join(x.split('-')[2:]))
model_names_list = df_2['model'].str.split('-').tolist()
# Remove all underscores from our 2d list
model_names_list = map2d(remove_underscore, model_names_list)
model_names = np.array(model_names_list)

# Splitting model name into more columns
df_2['train_data'] = np.char.add(model_names[:, 0], model_names[:, 1])
df_2['ratio'] = model_names[:, 2]
df_2['vision_model'] = model_names[:, 3]
df_2['method'] = model_names[:, 4]
df_2['keyword'] = model_names[:, 5]
df_2['AL'] = model_names[:, 6]
df_2['PL'] = model_names[:, 7]
df_2['vit_AL'] = model_names[:, 8]
df_2.drop('model', axis = 1, inplace = True)

cols = df_2.columns.tolist()
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
df_2['ratio'] = pd.to_numeric(df_2['ratio'])

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_2 = df_2.groupby(list(set(cols)-set(['value']))).agg({'value':['mean', 'std', 'count']})

In [4]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_2)

value  \
                                                                                                          mean   
metric             vit_AL AL    PL         vision_model keyword   ratio method dataset   train_data              
image_to_text_R@1  False  False None       RN50         none      0.1   base   RSICD     RSALL        4.658000   
                                                                               Sydney    RSALL        3.422000   
                                                                               UCM       RSALL        3.298000   
                                soft.image RN50         classname 0.1   ours   RSICD     RSALL        4.114000   
                                                                               Sydney    RSALL        2.536000   
                                                                               UCM       RSALL        2.428000   
                          True  None       RN50         none      0.1   base   RSICD     RSALL        3.110000   
                                                                               Sydney    RSALL        2.045000   
                                                                               UCM       RSALL        3.163333   
                   True   False soft.image RN50         classname 0.1   ours   RSICD     RSALL        3.692000   
                                                                               Sydney    RSALL        3.100000   
                                                                               UCM       RSALL        2.368000   
image_to_text_R@10 False  False None       RN50         none      0.1   base   RSICD     RSALL       29.726000   
                                                                               Sydney    RSALL       19.074000   
                                                                               UCM       RSALL       21.596000   
                                soft.image RN50         classname 0.1   ours   RSICD     RSALL       29.744000   
                                                                               Sydney    RSALL       17.344000   
                                                                               UCM       RSALL       17.154000   
                          True  None       RN50         none      0.1   base   RSICD     RSALL       20.293333   
                                                                               Sydney    RSALL       13.583333   
                                                                               UCM       RSALL       15.365000   
                   True   False soft.image RN50         classname 0.1   ours   RSICD     RSALL       26.984000   
                                                                               Sydney    RSALL       18.026000   
                                                                               UCM       RSALL       17.822000   
image_to_text_R@5  False  False None       RN50         none      0.1   base   RSICD     RSALL       17.660000   
                                                                               Sydney    RSALL       11.790000   
                                                                               UCM       RSALL       13.058000   
                                soft.image RN50         classname 0.1   ours   RSICD     RSALL       16.818000   
                                                                               Sydney    RSALL       10.502000   
                                                                               UCM       RSALL        9.868000   
                          True  None       RN50         none      0.1   base   RSICD     RSALL       11.701667   
                                                                               Sydney    RSALL        7.410000   
                                                                               UCM       RSALL        9.910000   
 